In [16]:
from nba_api.stats.endpoints import BoxScoreSummaryV2, BoxScoreAdvancedV2, LeagueGameLog
import pandas as pd
import json
import time

# Step 1: Fetch Game Data from NBA API
def get_game_data(season='2023-24'):
    """
    Fetch all game data for the specified NBA season and return a list of game IDs.
    """
    game_log = LeagueGameLog(season=season, timeout=10)
    game_data = game_log.get_data_frames()[0]
    return game_data[['GAME_ID', 'MATCHUP']]

# Step 2: Get Starters for Each Team
def get_starters(game_id):
    """
    Fetch the starters for both teams from a given game ID.
    """
    box_score = BoxScoreAdvancedV2(game_id=game_id)
    player_stats = box_score.get_data_frames()[0]
    
    # Filter players who started the game
    starters = player_stats[player_stats['START_POSITION'].notna()]
    
    team_0_starters = starters[starters['TEAM_ID'] == starters['TEAM_ID'].iloc[0]]['PLAYER_NAME'].tolist()
    team_1_starters = starters[starters['TEAM_ID'] != starters['TEAM_ID'].iloc[0]]['PLAYER_NAME'].tolist()
    print(f"Successfully fetched starters for game {game_id}")
    return team_0_starters, team_1_starters

# Step 3: Load Player Attributes from league.json
def load_player_attributes(filepath='league.json'):
    """
    Load player attributes from league.json and return as a dictionary for quick lookup.
    """
    with open(filepath) as f:
        player_data = json.load(f)
    return {player['name']: player for player in player_data}

# Step 4: Compute Average Attributes
def compute_team_averages(starters, player_attributes):
    """
    Compute average attributes for a list of starters.
    """
    attributes = [player_attributes[player] for player in starters if player in player_attributes]
    return pd.DataFrame(attributes).mean()

# Step 5: Determine Game Winner
def get_game_winner(game_id):
    """
    Fetch the winner of a game based on the final scores from DataFrame 5.
    """
    summary = BoxScoreSummaryV2(game_id=game_id, timeout=10)
    
    # Extract the correct dataframe (DataFrame 5)
    team_data = summary.get_data_frames()[5]
    
    # Check if 'PTS' exists in the dataframe
    if 'PTS' not in team_data.columns:
        raise ValueError(f"Column 'PTS' not found in team stats for game {game_id}.")
    
    # Extract scores
    team_0_score = team_data.iloc[0]['PTS']  # Assume row 0 is for Team 0
    team_1_score = team_data.iloc[1]['PTS']  # Assume row 1 is for Team 1
    
    # Determine winner
    return 0 if team_0_score > team_1_score else 1

# Step 6: Create DataFrame with Results
def create_game_dataframe(game_data, player_attributes):
    """
    Create a dataframe with average attributes for both teams and the game outcome.
    """
    rows = []
    game_count = 0  # To track batches of 5 games
    
    for index, row in game_data.iterrows():
        game_id = row['GAME_ID']
        try:
            team_0_starters, team_1_starters = get_starters(game_id)
            team_0_avg = compute_team_averages(team_0_starters, player_attributes)
            team_1_avg = compute_team_averages(team_1_starters, player_attributes)
            
            game_result = get_game_winner(game_id)
            
            combined = pd.concat([team_0_avg.add_prefix('team_0_'), team_1_avg.add_prefix('team_1_')])
            combined['winner'] = game_result
            rows.append(combined)
            print(f"Processed game {game_id}")
            
            # Increment game count and pause every 5 games
            game_count += 1
            if game_count % 5 == 0:
                print("Processed 5 games. Taking a 1-minute break...")
                time.sleep(60)  # 1-minute break
        except Exception as e:
            print(f"Error processing game {game_id}: {e}")
    
    return pd.DataFrame(rows)

# Step 7: Main Workflow
def main1():
    season = '2023-24'
    game_data = get_game_data(season=season)
    player_attributes = load_player_attributes('league.json')
    game_df = create_game_dataframe(game_data, player_attributes)
    game_df.to_csv('game_attributes.csv', index=False)
    print("Dataframe created and saved to game_attributes.csv")


main1()


Successfully fetched starters for game 0022300061
Processed game 0022300061
Successfully fetched starters for game 0022300062


/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will defaul

Processed game 0022300062
Successfully fetched starters for game 0022300062
Processed game 0022300062


/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()


Successfully fetched starters for game 0022300061
Processed game 0022300061


...

Processed game 0022301199
Processed 5 games. Taking a 1-minute break...
Successfully fetched starters for game 0022301200
Processed game 0022301200
Successfully fetched starters for game 0022301192


/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will defaul

Processed game 0022301192
Successfully fetched starters for game 0022301193
Processed game 0022301193


/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return pd.DataFrame(attributes).mean()
/var/folders/gv/mxw7p1nn5_b2n6n_7wkjz2sw0000gn/T/ipykernel_51356/3838135140.py:46: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will defaul

Successfully fetched starters for game 0022301189
Processed game 0022301189
Dataframe created and saved to game_attributes.csv
